In [1]:
'''
数据聚集
k-means
VectorAssmbler->StandardScaler->fit->transform
'''

'\n数据聚集\nk-means\nVectorAssmbler->StandardScaler->fit->transform\n'

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PySpark06").master("local[2]"). \
config("spark.sql.warehouse.dir","file:///E:/input/spark/warehouse").getOrCreate()

#data = spark.read.csv("file:///e:/Download/mllib/seeds_dataset.txt",inferSchema=True,header=True)

In [3]:
data_txt = spark.read.csv("file:///e:/Download/mllib/seeds_dataset.txt",inferSchema=True,sep='\t') # 文本文件
data_txt.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)



In [4]:
data_csv = spark.read.csv("file:///e:/Download/mllib/seeds_dataset.csv",inferSchema=True) # 文本文件
data_csv.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: integer (nullable = true)



In [5]:
data_csv.head(1)

[Row(_c0=15.26, _c1=14.84, _c2=0.871, _c3=5.763, _c4=3.312, _c5=2.221, _c6=5.22, _c7=1)]

In [6]:
#['area','perimeter','compactness','length_of_kernel','width_of_kernel','asymmetry_coefficient','length_of_froove','label']
data_csv.columns

['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6', '_c7']

In [7]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols=['_c0', '_c1', '_c2', '_c3', '_c4', '_c5', '_c6'],outputCol='features')

In [9]:
final_data = assembler.transform(data_csv) # 增加了一个向量features
final_data.printSchema

<bound method DataFrame.printSchema of DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: double, _c5: double, _c6: double, _c7: int, features: vector]>

In [10]:
from pyspark.ml.feature import StandardScaler # scala栏位

In [11]:
scaler = StandardScaler(inputCol='features',outputCol='scaledFeatures') # 标准缩放

In [12]:
scaler_model = scaler.fit(final_data) # 标准化，返回model

In [13]:
scaler_model

StandardScaler_4b62ac37ed10f6f9651e

In [14]:
final_data = scaler_model.transform(final_data) # 返回DataFrame

In [15]:
final_data.head(1)

[Row(_c0=15.26, _c1=14.84, _c2=0.871, _c3=5.763, _c4=3.312, _c5=2.221, _c6=5.22, _c7=1, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [16]:
kmeans = KMeans(featuresCol='scaledFeatures',predictionCol='prediction').setK(3).setSeed(42)

In [17]:
model = kmeans.fit(final_data) # fit返回模型

In [18]:
model.summary

In [19]:
model.clusterCenters()

[array([ 4.07497225, 10.14410142, 35.89816849, 11.80812742,  7.54416916,
         3.15411286, 10.38031464]),
 array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
         2.41585309, 12.29286107]),
 array([ 4.96198582, 10.97871333, 37.30930808, 12.44647267,  8.62880781,
         1.80062386, 10.41913733])]

In [20]:
wssse = model.computeCost(final_data)
wssse

428.60821613630515

In [21]:
results = model.transform(final_data) #返回新建prediction的DataFrame

In [22]:
results.columns

['_c0',
 '_c1',
 '_c2',
 '_c3',
 '_c4',
 '_c5',
 '_c6',
 '_c7',
 'features',
 'scaledFeatures',
 'prediction']

In [23]:
from collections import OrderedDict
cost = OrderedDict()
for i in range(2,8):
    kmeans = KMeans(featuresCol='scaledFeatures',predictionCol='prediction').setK(i).setSeed(1)
    model = kmeans.fit(final_data)
    wssse = model.computeCost(final_data)
    print("centers: %d, cost: %f" %(i,wssse))
    print(model.clusterCenters())
    cost[i]=wssse

centers: 2, cost: 656.793225
[array([ 4.42210624, 10.46640451, 36.50749337, 12.04573012,  7.98091715,
        2.56067167, 10.33364421]), array([ 6.20884577, 12.25651292, 37.43485358, 13.77282897,  9.67731721,
        2.29894074, 12.09236686])]
centers: 3, cost: 428.633351
[array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
        2.41585309, 12.29286107]), array([ 4.07135818, 10.14438097, 35.86461803, 11.81349589,  7.53471695,
        3.18317516, 10.39230304]), array([ 4.94114963, 10.95557919, 37.3028184 , 12.42383591,  8.60815545,
        1.80983779, 10.40657797])]
centers: 4, cost: 384.117920
[array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
        2.41585309, 12.29286107]), array([ 4.94744855, 10.96692617, 37.27127694, 12.44433954,  8.60597995,
        1.68529952, 10.42082516]), array([ 3.99907667, 10.09702376, 35.57069688, 11.80284016,  7.394573  ,
        3.00329767, 10.40661462]), array([ 4.44392161, 10.42255647, 37.06497816, 11.91982

In [24]:
cost

OrderedDict([(2, 656.793225326981),
             (3, 428.63335105690976),
             (4, 384.1179196509188),
             (5, 337.86050826590963),
             (6, 304.52114943408196),
             (7, 293.8692870931636)])

In [25]:
list = []
for dist in cost.values():
    list.append(dist)

In [26]:
import matplotlib.pyplot as plt
display(plt.plot(list))

In [27]:
import pandas as pd

In [28]:
#测试imblearn.combine.SMOTEENN
from imblearn.combine import SMOTEENN 

Using TensorFlow backend.
